## Generating data for ML

In [2]:
import pandas as pd
from collections import Counter
import random 
import ultimate
import numpy as np
# define card set
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
rank_values = {rank: i for i, rank in enumerate(ranks, start=2)}

deck = [{'rank': rank, 'suit': suit} for suit in suits for rank in ranks]

combinations = ["High Card", "One Pair", "Two Pair", "Three of a Kind", "Four of a Kind", 
                "Full House", "Straight", "Flush", "Straight Flush", "Royal Flush"]
combinations_values = {combination: i for i, combination in enumerate(combinations, start=1)}
# set ordered winning combinations
winning_hands = ["High Card", "One Pair", "Two Pair", "Three of a Kind", "Straight", "Flush", 
                "Full House", "Four of a Kind", "Straight Flush", "Royal Flush"]

winning_hand_ranks = {hand: i for i, hand in enumerate(winning_hands)}
#enumerate the deck
enumerated_deck = dict(enumerate(deck, start=1))
num_deck = list(range(1, 53))

In [2]:
# first round options
print(52*51*50*49 / 4)
# second round options
print(52*51*50*49*48*47*46 / 12)
# third round options
print(52*51*50*49*48*47*46*45*44 / 24)
enumerated_deck

1624350.0
56189515200.0
55627620048000.0


{1: {'rank': '2', 'suit': 'Hearts'},
 2: {'rank': '3', 'suit': 'Hearts'},
 3: {'rank': '4', 'suit': 'Hearts'},
 4: {'rank': '5', 'suit': 'Hearts'},
 5: {'rank': '6', 'suit': 'Hearts'},
 6: {'rank': '7', 'suit': 'Hearts'},
 7: {'rank': '8', 'suit': 'Hearts'},
 8: {'rank': '9', 'suit': 'Hearts'},
 9: {'rank': '10', 'suit': 'Hearts'},
 10: {'rank': 'J', 'suit': 'Hearts'},
 11: {'rank': 'Q', 'suit': 'Hearts'},
 12: {'rank': 'K', 'suit': 'Hearts'},
 13: {'rank': 'A', 'suit': 'Hearts'},
 14: {'rank': '2', 'suit': 'Diamonds'},
 15: {'rank': '3', 'suit': 'Diamonds'},
 16: {'rank': '4', 'suit': 'Diamonds'},
 17: {'rank': '5', 'suit': 'Diamonds'},
 18: {'rank': '6', 'suit': 'Diamonds'},
 19: {'rank': '7', 'suit': 'Diamonds'},
 20: {'rank': '8', 'suit': 'Diamonds'},
 21: {'rank': '9', 'suit': 'Diamonds'},
 22: {'rank': '10', 'suit': 'Diamonds'},
 23: {'rank': 'J', 'suit': 'Diamonds'},
 24: {'rank': 'Q', 'suit': 'Diamonds'},
 25: {'rank': 'K', 'suit': 'Diamonds'},
 26: {'rank': 'A', 'suit': 'Diamo

## For one player

In [2]:
# ante is 1, blind is 1, player bets 1 or 0 (return bet or 
# fold depending whether you will win or lose)
def decide_game_victor_and_calculate_rewards_for_bets(row):
    
    player_hand = [enumerated_deck[card] for card in row[0:7]]
    dealer_hand = [enumerated_deck[card] for card in row[2:]]

    player_combination = ultimate.get_best_hand(player_hand)
    dealer_combination = ultimate.get_best_hand(dealer_hand)

    player_rank = winning_hand_ranks[player_combination]
    dealer_rank = winning_hand_ranks[dealer_combination]

    victor = 0 # 0 = dealer, 1 = player
    
    if player_rank > dealer_rank:
        victor = 1
    elif player_rank == dealer_rank:
        result = ultimate.decider(player_combination, player_hand, 
                                  dealer_combination, dealer_hand)
        if result == "player":
            victor = 1
        elif result == "dealer":	
            victor = 0
        else:
            victor = random.randint(0, 1)	# need to decide about this
    else:
        victor = 0

    # check if ante is valid
    dealer_has_something = ultimate.dealer_has_pair_or_better(dealer_hand[:2], dealer_hand[2:])
    blind_won = ultimate.has_blind(1, player_combination) - 1 #how much blind got us

    # calculate rewards for first and second rounds (in third victory is already bet, defeat is fold)
    first_round = 0
    second_round = 0
    third_round = 0

    # if its a draw, we leave both values at zero
    if victor == 1:
        first_round = 4 + blind_won + (1 if dealer_has_something else 0)
        second_round = 2 + blind_won + (1 if dealer_has_something else 0)
        third_round = 1 + blind_won + (1 if dealer_has_something else 0)
    elif victor == 0:
        first_round = -6 + (1 if not dealer_has_something else 0)
        second_round = -4 + (1 if not dealer_has_something else 0)
        third_round = -3 + (1 if not dealer_has_something else 0)
    
    return [third_round, first_round, second_round]

# function for conversion back to cards for checking
def convert_to_cards(row):
    cards = row[:9]
    output = [(enumerated_deck[card]["rank"], enumerated_deck[card]["suit"])  for card in cards]
    return output


In [ ]:
# full dataset
hands = 2000
multiplier_D = 15
multiplier_1 = 15
multiplier_2 = 15

header = ["C1", "C2", "R1", "R2", "R3", "R4", "R5", "D1", "D2"]
rows = []

for A in range(hands):
    if A % 1000 == 0:
        print(f"Hand {A}")
    player = random.sample(num_deck, 2)
    for _ in range(multiplier_1):
        remaining_deck = [card for card in num_deck if card not in player]
        flop = random.sample(remaining_deck, 3)
        for _ in range(multiplier_2):
            used = set(player + flop)
            river = random.sample([card for card in num_deck if card not in used], 2)
            for _ in range(multiplier_D):
                used2 = set(player + flop + river)
                dealer = random.sample([card for card in num_deck if card not in used2], 2)
                rows.append(player + flop + river + dealer)
full_data_set = pd.DataFrame(rows, columns=header)
print("generated variables")
full_data_set[["Q3", "Q1", "Q2"]] = full_data_set.apply(decide_game_victor_and_calculate_rewards_for_bets, axis=1, result_type="expand")
print("rewards calculated")
exp_values_Q1 = full_data_set.groupby(["C1", "C2"], sort=False).mean().reset_index().drop(columns=["R1", "R2", "R3", "R4", "R5", "D1", "D2", "Q3", "Q2"])
exp_values_Q2 = full_data_set.groupby(["C1", "C2", "R1", "R2", "R3"], sort=False).mean().reset_index().drop(columns=["R4", "R5", "D1", "D2", "Q3", "Q1"])
exp_values_Q3 = full_data_set.groupby(["C1", "C2", "R1", "R2", "R3", "R4", "R5"], sort=False).mean().reset_index().drop(columns=["D1", "D2", "Q2", "Q1"])
print("mean")
exp_values_Q1["Y1"] = exp_values_Q1["Q1"].apply(lambda x: 1 if x > 0 else 0)
exp_values_Q2["Y2"] = exp_values_Q2["Q2"].apply(lambda x: 1 if x > 0 else 0)
exp_values_Q3["Y3"] = exp_values_Q3["Q3"].apply(lambda x: 1 if x > 0 else 0)
print("y decided")
data_for_first_round = exp_values_Q1.drop(columns=["Q1"])
data_for_second_round = exp_values_Q2.drop(columns=["Q2"])
data_for_third_round = exp_values_Q3.drop(columns=["Q3"])
#data_for_second = full_data_set[::10].drop(columns=["R4", "R5"]) #get without last two cards
#data_for_first = data_for_second[::10].drop(columns=["R1", "R2", "R3"])  #get only first two cards
#data_for_first

#testing
pd.set_option('display.max_rows', 20)  # Show up to 100 rows
#test_copy = full_data_set.copy()
#test_copy[["C1", "C2", "R1", "R2", "R3", "R4", "R5", "D1", "D2"]] = test_copy.apply(convert_to_cards, 
#                                                                                    axis=1, result_type="expand")
#test_copy,
np.save("data_for_first_round.npy", data_for_first_round.to_numpy())
np.save("data_for_second_round.npy", data_for_second_round.to_numpy())
np.save("data_for_third_round.npy", data_for_third_round.to_numpy())
#print(data_for_first_round)
#print(data_for_second_round)
#print(data_for_third_round)

Hand 0
Hand 1000
generated variables
rewards calculated
mean
y decided


## Second approach for one player

In [4]:
# ante is 1, blind is 1, player bets 1 or 0 (return bet or 
# fold depending whether you will win or lose)
def decide_game_victor(row):
    
    player_hand = [enumerated_deck[card] for card in row[0:7]]
    dealer_hand = [enumerated_deck[card] for card in row[2:]]

    player_combination = ultimate.get_best_hand(player_hand)
    dealer_combination = ultimate.get_best_hand(dealer_hand)

    player_rank = winning_hand_ranks[player_combination]
    dealer_rank = winning_hand_ranks[dealer_combination]

    victor = 0 # 0 = dealer, 1 = player
    
    if player_rank > dealer_rank:
        victor = 1
    elif player_rank == dealer_rank:
        result = ultimate.decider(player_combination, player_hand, 
                                  dealer_combination, dealer_hand)
        if result == "player":
            victor = 1
        elif result == "dealer":	
            victor = 0
        else:
            victor = random.randint(0, 1)	# need to decide about this
    else:
        victor = 0

    return victor

# function for conversion back to cards for checking
def convert_to_cards(row):
    cards = row[:9]
    output = [(enumerated_deck[card]["rank"], enumerated_deck[card]["suit"])  for card in cards]
    return output


In [11]:
# full dataset
hands = 2000
multiplier_D = 15
multiplier_1 = 20
multiplier_2 = 15

header = ["C1", "C2", "R1", "R2", "R3", "R4", "R5", "D1", "D2"]
rows = []

for A in range(hands):
    if A % 1000 == 0:
        print(f"Hand {A}")
    player = random.sample(num_deck, 2)
    for _ in range(multiplier_1):
        remaining_deck = [card for card in num_deck if card not in player]
        flop = random.sample(remaining_deck, 3)
        for _ in range(multiplier_2):
            used = set(player + flop)
            river = random.sample([card for card in num_deck if card not in used], 2)
            for _ in range(multiplier_D):
                used2 = set(player + flop + river)
                dealer = random.sample([card for card in num_deck if card not in used2], 2)
                rows.append(player + flop + river + dealer)

full_data_set = pd.DataFrame(rows, columns=header)
print("generated variables")
full_data_set["V"] = full_data_set.apply(decide_game_victor, axis=1, result_type="expand")
print("rewards calculated")
exp_values_Q1 = full_data_set.groupby(["C1", "C2"], sort=False).mean().reset_index().drop(columns=["R1", "R2", "R3", "R4", "R5", "D1", "D2"])
exp_values_Q2 = full_data_set.groupby(["C1", "C2", "R1", "R2", "R3"], sort=False).mean().reset_index().drop(columns=["R4", "R5", "D1", "D2"])
exp_values_Q3 = full_data_set.groupby(["C1", "C2", "R1", "R2", "R3", "R4", "R5"], sort=False).mean().reset_index().drop(columns=["D1", "D2"])
print("mean")
#exp_values_Q1["Y1"] = exp_values_Q1["V"].apply(lambda x: 1 if x > 0.5 else 0)
#exp_values_Q2["Y2"] = exp_values_Q2["V"].apply(lambda x: 1 if x > 0.5 else 0)
#exp_values_Q3["Y3"] = exp_values_Q3["V"].apply(lambda x: 1 if x > 0.5 else 0)
#print("y decided")
#data_for_first_round = exp_values_Q1.drop(columns=["V"])
#data_for_second_round = exp_values_Q2.drop(columns=["V"])
#data_for_third_round = exp_values_Q3.drop(columns=["V"])
data_for_first_round = exp_values_Q1
data_for_second_round = exp_values_Q2
data_for_third_round = exp_values_Q3

pd.set_option('display.max_rows', 20)  # Show up to 100 rows

np.save("data_for_first_round.npy", data_for_first_round.to_numpy())
np.save("data_for_second_round.npy", data_for_second_round.to_numpy())
np.save("data_for_third_round.npy", data_for_third_round.to_numpy())
#print(data_for_first_round)
#print(data_for_second_round)
#print(data_for_third_round)

Hand 0
Hand 1000
generated variables
rewards calculated
mean


## For n players

In [11]:
# ante is 1, blind is 1, player bets 1 or 0 (return bet or 
# fold depending whether you will win or lose)
def decide_game_victor_and_calculate_rewards_for_bets_n(row, n):
    
    player_hand = [enumerated_deck[card] for card in row[["C1_P1", "C2_P1", "R1", "R2", "R3", "R4", "R5"]]]
    dealer_hand = [enumerated_deck[card] for card in row[["R1", "R2", "R3", "R4", "R5", "D1", "D2"]]]

    player_combination = ultimate.get_best_hand(player_hand)
    dealer_combination = ultimate.get_best_hand(dealer_hand)

    victor = 0 # 0 = dealer, 1 = player
    
    if winning_hands.index(player_combination) > winning_hands.index(dealer_combination):
        victor = 1
    elif winning_hands.index(player_combination) == winning_hands.index(dealer_combination):
        result = ultimate.decider(player_combination, player_hand, 
                                  dealer_combination, dealer_hand)
        if result == "player":
            victor = 1
        elif result == "dealer":	
            victor = 0
        else:
            victor = 2	# need to decide about this
    else:
        victor = 0

    # check if ante is valid
    dealer_has_something = ultimate.dealer_has_pair_or_better(dealer_hand[:2], dealer_hand[2:])
    blind_won = ultimate.has_blind(1, player_combination) - 1 #how much blind got us

    # calculate rewards for first and second rounds (in third victory is already bet, defeat is fold)
    first_round = 0
    second_round = 0

    # if its a draw, we leave both values at zero
    if victor == 1:
        first_round = 4 + blind_won + (1 if dealer_has_something else 0)
        second_round = 2 + blind_won + (1 if dealer_has_something else 0)
    elif victor == 0:
        first_round = -6 + (1 if not dealer_has_something else 0)
        second_round = -4 + (1 if not dealer_has_something else 0)
    
    return [victor, first_round, second_round]

# function for conversion back to cards for checking
def convert_to_cards_n(row, n):
    cards = row[:(7 + 2 * n)]
    output = [(enumerated_deck[card]["rank"], enumerated_deck[card]["suit"])  for card in cards]
    return output


In [ ]:
import random
import numpy as np
import pandas as pd

# -------------------------
# Parameters
# -------------------------
hands = 2000
multiplier_D = 15
multiplier_1 = 15
multiplier_2 = 15
n = 6  # total players; Player 1 is evaluated

# num_deck must exist (e.g., list(range(52)))
# decide_game_victor_and_calculate_rewards_for_bets(row) must exist and return (Q3, Q1, Q2)

header = []  # Cards j Player i
for i in range(n):
    header += [f"C1_P{i+1}", f"C2_P{i+1}"]
header += ["R1", "R2", "R3", "R4", "R5", "D1", "D2"]
print(header)

rows = []

for A in range(hands):
    if A % 1000 == 0:
        print(f"Hand {A}")

    # Player 1 (hero)
    p1 = random.sample(num_deck, 2)

    # Flops per hero
    remaining_after_p1 = [card for card in num_deck if card not in p1]
    for _ in range(multiplier_1):
        flop = random.sample(remaining_after_p1, 3)

        # (turn, river) per flop
        for _ in range(multiplier_2):
            used = set(p1 + flop)
            r4_r5 = random.sample([card for card in num_deck if card not in used], 2)
            board = flop + r4_r5  # R1..R5

            # Opponent lineups per full board
            for _ in range(multiplier_D):
                used2 = set(p1 + board)
                deck_for_opps = [card for card in num_deck if card not in used2]

                # Need hands for players 2..n
                cards_needed = 2 * (n - 1)
                opp_flat = random.sample(deck_for_opps, cards_needed)  # [C1_P2, C2_P2, ..., C1_Pn, C2_Pn]

                # Reserve D1, D2 from remaining (to preserve your schema)
                used3 = set(p1 + board + opp_flat)
                deck_for_D = [card for card in num_deck if card not in used3]
                # If your reward func needs D1/D2, they’ll be valid; else harmless
                if len(deck_for_D) >= 2:
                    d1_d2 = random.sample(deck_for_D, 2)
                else:
                    d1_d2 = [-1, -1]  # edge fallback if parameters exhaust the deck

                # Assemble row: [P1, P2..Pn, R1..R5, D1, D2]
                players_flat = p1[:] + opp_flat
                row = players_flat + board + d1_d2
                rows.append(row)

full_data_set = pd.DataFrame(rows, columns=header)
print("generated variables")
full_data_set[["Q3", "Q1", "Q2"]] = full_data_set.apply(
    decide_game_victor_and_calculate_rewards_for_bets_n, axis=1, result_type="expand"
)
print("rewards calculated")

q1_keys = ["C1_P1", "C2_P1"]
q2_keys = ["C1_P1", "C2_P1", "R1", "R2", "R3"]
q3_keys = ["C1_P1", "C2_P1", "R1", "R2", "R3", "R4", "R5"]

exp_values_Q1 = (
    full_data_set.groupby(q1_keys, sort=False).mean(numeric_only=True).reset_index()
    .drop(columns=[c for c in ["R1","R2","R3","R4","R5","D1","D2","Q3","Q2"] if c in full_data_set.columns])
)
exp_values_Q2 = (
    full_data_set.groupby(q2_keys, sort=False).mean(numeric_only=True).reset_index()
    .drop(columns=[c for c in ["R4","R5","D1","D2","Q3","Q1"] if c in full_data_set.columns])
)
exp_values_Q3 = (
    full_data_set.groupby(q3_keys, sort=False).mean(numeric_only=True).reset_index()
    .drop(columns=[c for c in ["D1","D2","Q2","Q1"] if c in full_data_set.columns])
)

print("mean")

exp_values_Q1["Y1"] = (exp_values_Q1["Q1"] > 0).astype(int)
exp_values_Q2["Y2"] = (exp_values_Q2["Q2"] > 0).astype(int)
exp_values_Q3["Y3"] = (exp_values_Q3["Q3"] > 0).astype(int)
print("y decided")

data_for_first_round  = exp_values_Q1.drop(columns=["Q1"])
data_for_second_round = exp_values_Q2.drop(columns=["Q2"])
data_for_third_round  = exp_values_Q3.drop(columns=["Q3"])

pd.set_option('display.max_rows', 20)  # Show up to 20 rows

np.save("data_for_first_round_n.npy",  data_for_first_round.to_numpy())
np.save("data_for_second_round_n.npy", data_for_second_round.to_numpy())
np.save("data_for_third_round_n.npy",  data_for_third_round.to_numpy())


## Second approach for n players

In [ ]:
# ante is 1, blind is 1, player bets 1 or 0 (return bet or 
# fold depending whether you will win or lose)
def decide_game_victor_n(row):
    
    player_hand = [enumerated_deck[card] for card in row[["C1_P1", "C2_P1", "R1", "R2", "R3", "R4", "R5"]]]
    dealer_hand = [enumerated_deck[card] for card in row[["R1", "R2", "R3", "R4", "R5", "D1", "D2"]]]

    player_combination = ultimate.get_best_hand(player_hand)
    dealer_combination = ultimate.get_best_hand(dealer_hand)

    player_rank = winning_hand_ranks[player_combination]
    dealer_rank = winning_hand_ranks[dealer_combination]

    victor = 0 # 0 = dealer, 1 = player
    
    if player_rank > dealer_rank:
        victor = 1
    elif player_rank == dealer_rank:
        result = ultimate.decider(player_combination, player_hand, 
                                  dealer_combination, dealer_hand)
        if result == "player":
            victor = 1
        elif result == "dealer":	
            victor = 0
        else:
            victor = random.randint(0, 1)	# need to decide about this
    else:
        victor = 0

    return victor

In [ ]:
import random
import numpy as np
import pandas as pd

hands = 2
multiplier_D = 1
multiplier_1 = 1
multiplier_2 = 1
n = 2  # total players; Player 1 is the one we evaluate V for

header = []  # Cards j Player i
for i in range(n):
    header += ["C1_P" + str(i + 1), "C2_P" + str(i + 1)]
header += ["R1", "R2", "R3", "R4", "R5", "D1", "D2"]
print(header)
pd.set_option("display.max_colwidth", None)  # Show full cell contents
pd.set_option("display.width", None)         # Don't wrap based on console width
rows = []

for A in range(hands):
    if A % 1000 == 0:
        print(f"Hand {A}")

    # Player 1 (hero)
    p1 = random.sample(num_deck, 2)

    # Flops per hero
    remaining_after_p1 = [card for card in num_deck if card not in p1]
    for _ in range(multiplier_1):
        flop = random.sample(remaining_after_p1, 3)

        # (turn, river) per flop
        for _ in range(multiplier_2):
            used = set(p1 + flop)
            r4_r5 = random.sample([card for card in num_deck if card not in used], 2)
            board = flop + r4_r5  # [R1..R5]

            # Opponent lineups per full board
            for _ in range(multiplier_D):
                used2 = set(p1 + board)
                deck_for_opps = [card for card in num_deck if card not in used2]

                # Need hands for players 2..n
                cards_needed = 2 * (n - 1)
                opp_flat = random.sample(deck_for_opps, cards_needed)

                # Now reserve D1,D2 from remaining (just to preserve schema)
                used3 = set(p1 + board + opp_flat)
                deck_for_D = [card for card in num_deck if card not in used3]
                # If you don't actually use D1/D2 in your victor fn, they're harmless
                if len(deck_for_D) >= 2:
                    d1_d2 = random.sample(deck_for_D, 2)
                else:
                    # Fallback if deck exhausted due to parameter choices
                    d1_d2 = [-1, -1]

                # Assemble row as [P1, P2, ..., Pn, board, D1, D2]
                players_flat = p1[:]  # P1
                players_flat += opp_flat  # P2..Pn already flat list [C1_P2, C2_P2, ..., C1_Pn, C2_Pn]
                row = players_flat + board + d1_d2
                rows.append(row)

full_data_set = pd.DataFrame(rows, columns=header)
print("generated variables")
full_data_set["V"] = full_data_set.apply(decide_game_victor_n, axis=1, result_type="expand")
print("rewards calculated")

q1_keys = ["C1_P1", "C2_P1"]
q2_keys = ["C1_P1", "C2_P1", "R1", "R2", "R3"]
q3_keys = ["C1_P1", "C2_P1", "R1", "R2", "R3", "R4", "R5"]

exp_values_Q1 = full_data_set.groupby(q1_keys, sort=False).mean(numeric_only=True).reset_index()
drop_cols_q1 = [c for c in exp_values_Q1.columns if c not in (q1_keys + ["V"])]
exp_values_Q1 = exp_values_Q1.drop(columns=drop_cols_q1)

exp_values_Q2 = full_data_set.groupby(q2_keys, sort=False).mean(numeric_only=True).reset_index()
drop_cols_q2 = [c for c in exp_values_Q2.columns if c not in (q2_keys + ["V"])]
exp_values_Q2 = exp_values_Q2.drop(columns=drop_cols_q2)

exp_values_Q3 = full_data_set.groupby(q3_keys, sort=False).mean(numeric_only=True).reset_index()
drop_cols_q3 = [c for c in exp_values_Q3.columns if c not in (q3_keys + ["V"])]
exp_values_Q3 = exp_values_Q3.drop(columns=drop_cols_q3)

print("mean")

# Keep full columns (like you do) or strip V later if needed
data_for_first_round  = exp_values_Q1
data_for_second_round = exp_values_Q2
data_for_third_round  = exp_values_Q3

pd.set_option('display.max_rows', 20)

np.save("data_for_first_round_n.npy",  data_for_first_round.to_numpy())
np.save("data_for_second_round_n.npy", data_for_second_round.to_numpy())
np.save("data_for_third_round_n.npy",  data_for_third_round.to_numpy())
print(data_for_first_round)
print(data_for_second_round)
print(data_for_third_round)


['C1_P1', 'C2_P1', 'C1_P2', 'C2_P2', 'R1', 'R2', 'R3', 'R4', 'R5', 'D1', 'D2']
Hand 0
generated variables
rewards calculated
mean
   C1_P1  C2_P1    V
0     17     38  1.0
1     19     49  1.0
   C1_P1  C2_P1  R1  R2  R3    V
0     17     38  23   4  16  1.0
1     19     49   4  20  36  1.0
0    [(5, Diamonds), (K, Clubs), (J, Diamonds), (5, Hearts), (4, Diamonds), (2, Clubs), (6, Spades), (2, Hearts)]
1      [(7, Diamonds), (J, Spades), (5, Hearts), (8, Diamonds), (J, Clubs), (Q, Hearts), (4, Clubs), (2, Hearts)]
dtype: object
